In [22]:
from tkinter import *
from PIL import Image, ImageTk
import cv2
import numpy as np
from threading import Thread

def create_image(root, filename, x=0, y=0, width=None, height=None, bg=None):
    """
    Создает и размещает изображение в окне Tkinter
    
    Параметры:
        root: окно Tkinter (Tk() или Frame)
        filename: имя файла изображения (например 'tank.png')
        x, y: координаты размещения (по умолчанию 0, 0)
        width: желаемая ширина (None - оригинальный размер)
        height: желаемая высота (None - оригинальный размер)
        bg: цвет фона (None - прозрачный)
    
    Возвращает:
        Label: созданный виджет с изображением
    """
    # Загружаем изображение через Pillow
    pil_image = Image.open(filename)
    
    # Изменяем размер если нужно
    if width or height:
        original_width, original_height = pil_image.size
        
        # Если указана только одна размерность, сохраняем пропорции
        if not width:
            ratio = height / original_height
            width = int(original_width * ratio)
        elif not height:
            ratio = width / original_width
            height = int(original_height * ratio)
            
        pil_image = pil_image.resize((width, height), Image.LANCZOS)
    
    # Конвертируем в формат Tkinter
    tk_image = ImageTk.PhotoImage(pil_image)
    
    # Создаем Label с изображением
    label = Label(root, image=tk_image, bg=bg)
    label.image = tk_image  # Сохраняем ссылку на изображение!
    label.place(x=x, y=y)
    
    return label


class CameraWidget:
    def __init__(self, parent, x=0, y=0):
        self.parent = parent
        self.camera_active = False
        self.video_capture = None
        
        # Создаем фрейм для камеры
        self.camera_frame = Frame(parent, bg='green')
        self.camera_frame.place(x=x, y=y)
        
        # Метка для вывода изображения
        self.camera_label = Label(self.camera_frame, bg='black')
        self.camera_label.pack()
        
        # Кнопка управления камерой
        self.btn_camera = Button(self.camera_frame, 
                               text='Включить камеру', 
                               command=self.toggle_camera,
                               font=('Arial', 10),
                               bg='blue',
                               fg='white')
        self.btn_camera.pack(pady=5)
    
    def toggle_camera(self):
        """Включение/выключение камеры"""
        if not self.camera_active:
            self.start_camera()
        else:
            self.stop_camera()
    
    def start_camera(self):
        """Запуск камеры"""
        self.camera_active = True
        self.btn_camera.config(text='Выключить камеру', bg='red')
        self.video_capture = cv2.VideoCapture(0)
        Thread(target=self.update_frame, daemon=True).start()
    
    def stop_camera(self):
        """Остановка камеры"""
        self.camera_active = False
        self.btn_camera.config(text='Включить камеру', bg='blue')
        if self.video_capture:
            self.video_capture.release()
        self.camera_label.config(image='')
    
    def update_frame(self):
        """Обновление кадров"""
        while self.camera_active:
            ret, frame = self.video_capture.read()
            if ret:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = cv2.resize(frame, (320, 240))
                img = Image.fromarray(frame)
                imgtk = ImageTk.PhotoImage(image=img)
                
                self.camera_label.imgtk = imgtk
                self.camera_label.config(image=imgtk)
                
                self.parent.update()
                cv2.waitKey(30)
            else:
                break

class Signals:
    def __init__(self, root):
        self.root = root
        self.current_dron = None  # Текущее изображение дрона
        self.dron_img = self._load_image()  # Загружаем изображение один раз

    def _load_image(self):
        """Загружает и подготавливает изображение дрона"""
        try:
            img = Image.open('dron_atak.png')
            img = img.resize((70, 70), Image.LANCZOS)  # Фиксированный размер
            return ImageTk.PhotoImage(img)
        except Exception as e:
            print(f"Ошибка загрузки изображения дрона: {e}")
            return None

    def dron_direction(self, direction):
        """Отображает атаку дрона с указанного направления"""
        # Удаляем предыдущее изображение
        if self.current_dron:
            self.current_dron.destroy()

        # Позиции для разных направлений
        positions = {
            'Front': {'x': 240, 'y': 160},
            'Right': {'x': 350, 'y': 320},
            'Left': {'x': 120, 'y': 320},
            'Back': {'x': 250, 'y': 500},
            'Up': {'x': 200, 'y': 100}
        }

        # Создаем новое изображение дрона
        if direction in positions and self.dron_img:
            pos = positions[direction]
            self.current_dron = Label(self.root, image=self.dron_img, bg='red')
            self.current_dron.place(x=pos['x'], y=pos['y'])


# Основное окно
root = Tk()
root.geometry('600x600')
root.resizable(False, False)
root.title('Система Щит с камерой')
root.config(bg='green')

# Виджет камеры
camera = CameraWidget(root, x=20, y=20)

# Кнопка системы (пример)
btn_system = Button(root, 
                   text='Системная кнопка', 
                   font=('Arial', 14),
                   bg='navy',
                   fg='white')
btn_system.place(x=0, y=50)

# Изображение танка с размещением через place
tank = create_image(root, 'tank.png', x=150, y=150, width=300, bg='green')


signals = Signals(root)
signals.dron_direction('Front')

root.mainloop()